# <h1 align="center"><font color="red">Qwen2-VL</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Links de estudo e teste de Qwen2-VL:

* Para testar no `Hugging Face`: [Qwen2-VL-Max é o nome da Interface no Gradio](https://huggingface.co/spaces/Qwen/Qwen2-VL) 

`O nome do modelo 'qwen-vl-max-0809' é a identidade de 'Qwen2-VL-72B'.`

* [GitHub: Qwen2-VL](https://github.com/QwenLM/Qwen2-VL)

* [Qwen's collections](https://huggingface.co/collections/Qwen/qwen2-vl-66cee7455501d7126940800d)

In [1]:
#FILE_NAME = "/home/eddygiusepe/2_EddyGiusepe_Estudo/Exploring_the_World_of_Programming_with_Python/41_Multimodal_Embeddings_images-audio-video/images/SIEMENS.jpeg"
FILE_NAME ="/home/eddygiusepe/2_EddyGiusepe_Estudo/Exploring_the_World_of_Programming_with_Python/41_Multimodal_Embeddings_images-audio-video/images/placa_motor_monofasico.JPG"

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import json

model_name = "Qwen/Qwen2-VL-2B-Instruct" # Qwen/Qwen2-VL-2B-Instruct ou   Qwen/Qwen2-VL-7B-Instruct    ou     Qwen/Qwen2-VL-72B-Instruct
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(
    model_name
)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Verifique o tipo de dado do primeiro parâmetro do modelo:
first_param = next(model.parameters())

first_param.dtype

torch.bfloat16

In [4]:
messages = [{
        "role": "system",
        "content": """Você é um experto em Análise de Vibração, Manutenção preditiva
                      e monitoramento de ativos industriais."""
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": FILE_NAME,
                "resized_height": 696,
                "resized_width": 943,
            },
            {
                "type": "text",
                "text": """
                        Extraia, APENAS, o tipo de motor.
                        """
            }
        ]
    }
]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=True)

output_text


['O tipo de motor é ALY711-2 B5.']

In [5]:
print(output_text[0])

O tipo de motor é ALY711-2 B5.
